In [1]:
from pylibfreenect2 import Freenect2, SyncMultiFrameListener
from pylibfreenect2 import FrameType, Registration, Frame
from pylibfreenect2 import CpuPacketPipeline

In [2]:
dp = {'cx': 253.314697265625,
 'cy': 207.460693359375,
 'fx': 365.63720703125,
 'fy': 365.63720703125,
 'k1': 0.09617280960083008,
 'k2': -0.2692984938621521,
 'k3': 0.08770948648452759,
 'p1': 0.0,
 'p2': 0.0}

rp = {'cx': 959.5,
 'cy': 539.5,
 'fx': 1081.3720703125,
 'fy': 1081.3720703125,
 'mx_x0y0': 0.1461970955133438,
 'mx_x0y1': 0.003376332111656666,
 'mx_x0y2': 0.0001174572971649468,
 'mx_x0y3': 4.1423638322157785e-05,
 'mx_x1y0': 0.6362712979316711,
 'mx_x1y1': -6.082074833102524e-06,
 'mx_x1y2': 0.0003575267910491675,
 'mx_x2y0': 0.00016135470650624484,
 'mx_x2y1': 2.191878047597129e-05,
 'mx_x3y0': 0.0005048222956247628,
 'my_x0y0': 0.005415176972746849,
 'my_x0y1': 0.6362615823745728,
 'my_x0y2': -9.692917228676379e-05,
 'my_x0y3': 0.0006501250900328159,
 'my_x1y0': -0.0037287920713424683,
 'my_x1y1': 3.459840081632137e-05,
 'my_x1y2': 2.968754961329978e-05,
 'my_x2y0': -0.00010923569789156318,
 'my_x2y1': 0.00040543320938013494,
 'my_x3y0': -8.966947007138515e-07,
 'shift_d': 863.0,
 'shift_m': 52.0}

from easydict import EasyDict

depth_p = EasyDict(dp)
rgb_p = EasyDict(rp)

In [ ]:
pipeline = CpuPacketPipeline()
fn = Freenect2()

In [ ]:
device = fn.openDefaultDevice(pipeline=pipeline)

In [ ]:
def distort(mx, my, depth_p):
    
    dx = (float(mx) - depth_p.cx) / depth_p.fx;
    dy = (float(my) - depth_p.cy) / depth_p.fy;
    dx2 = dx * dx;
    dy2 = dy * dy;
    r2 = dx2 + dy2;
    dxdy2 = 2 * dx * dy;
    kr = 1 + ((depth_p.k3 * r2 + depth_p.k2) * r2 + depth_p.k1) * r2;
    x = depth_p.fx * (dx * kr + depth_p.p2 * (r2 + 2 * dx2) + depth_p.p1 * dxdy2) + depth_p.cx;
    y = depth_p.fy * (dy * kr + depth_p.p1 * (r2 + 2 * dy2) + depth_p.p2 * dxdy2) + depth_p.cy;
    
    return x, y

depth_q = 0.01
color_q = 0.002199

def depth_to_color(mx, my, depth_p, rgb_p):
    mx = (mx - depth_p.cx) * depth_q
    my = (my - depth_p.cy) * depth_q

    wx = (mx * mx * mx * rgb_p.mx_x3y0) + (my * my * my * rgb_p.mx_x0y3) + \
    (mx * mx * my * rgb_p.mx_x2y1) + (my * my * mx * rgb_p.mx_x1y2) + \
    (mx * mx * rgb_p.mx_x2y0) + (my * my * rgb_p.mx_x0y2) + (mx * my * rgb_p.mx_x1y1) + \
    (mx * rgb_p.mx_x1y0) + (my * rgb_p.mx_x0y1) + (rgb_p.mx_x0y0)

    wy = (mx * mx * mx * rgb_p.my_x3y0) + (my * my * my * rgb_p.my_x0y3) + \
    (mx * mx * my * rgb_p.my_x2y1) + (my * my * mx * rgb_p.my_x1y2) + \
    (mx * mx * rgb_p.my_x2y0) + (my * my * rgb_p.my_x0y2) + (mx * my * rgb_p.my_x1y1) + \
    (mx * rgb_p.my_x1y0) + (my * rgb_p.my_x0y1) + (rgb_p.my_x0y0)

    rx = (wx / (rgb_p.fx * color_q)) - (rgb_p.shift_m / rgb_p.shift_d)
    ry = (wy / color_q) + rgb_p.cy
    
    return rx, ry

def getRegistrationMap(depth_p, rgb_p):
    import numpy as np
    distort_map = np.empty((512 * 424))
    depth_to_color_map_x = np.empty((512 * 424))
    depth_to_color_map_y = np.empty((512 * 424))
    depth_to_color_map_yi = np.empty((512 * 424))
    
    for y in range(424):
        for x in range(512):
            index = y * 512 + x
            mx, my = distort(x, y, depth_p)
            ix = int(round(mx))
            iy = int(round(my))
            if (ix < 0 or ix >= 512 or iy < 0 or iy >= 424):
                distort_index = -1
            else:
                distort_index = iy * 512 + ix
            
            distort_map[index] = distort_index
            
            rx, ry = depth_to_color(x, y, depth_p, rgb_p)
            depth_to_color_map_x[index] = rx
            depth_to_color_map_y[index] = ry
            depth_to_color_map_yi[index] = int(round(ry))
    
    map_dist = distort_map
    map_x = depth_to_color_map_x
    map_y = depth_to_color_map_y
    map_yi = depth_to_color_map_yi
    
    return map_dist, map_x, map_y, map_yi

In [ ]:
depth_p = device.getIrCameraParams()
rgb_p = device.getColorCameraParams()

In [3]:
import numpy as np
import os

In [10]:
map_dir = '/home/commaai-03/Test/tmp'
map_list = [os.path.join(map_dir, 'map%d.txt' % i) for i in range(1, 5)]

map1 = np.loadtxt(map_list[0])
map2 = np.loadtxt(map_list[1])
map3 = np.loadtxt(map_list[2])
map4 = np.loadtxt(map_list[3])

In [5]:
tmp_reshape = lambda mapx: np.reshape(mapx, (424, 512))
map1 = tmp_reshape(map1)
map2 = tmp_reshape(map2)
map3 = tmp_reshape(map3)
map4 = tmp_reshape(map4)

In [ ]:
map2

In [ ]:
map3

In [ ]:
map4

In [6]:
import cv2

color = '/home/commaai-03/Test/tmp/data/1589877210.962524156.png'
depth = '/home/commaai-03/Test/tmp/data/1589877210.962524156_.png'

color = cv2.imread(color)
depth = cv2.imread(depth, cv2.IMREAD_ANYDEPTH)

In [7]:
depth_flatten = depth.flatten()
color_flatten = color.reshape(-1, 3)

In [ ]:
color_flatten.shape

In [8]:
size_depth = 512 * 424
size_color = 1920 * 1080
color_cx = rgb_p.cx + 0.5

depth_to_c_off = np.empty((size_depth))
map_c_off = depth_to_c_off

# Filter
filter_map = np.empty((size_color + 1920 * 2))
p_filter_map = filter_map

In [11]:
for i in range(size_depth):
    index = int(map1[i])
    if index < 0:
        map_c_off[i] = -1
        continue
    z = depth_flatten[index]
    
    if z <= 0.:
        map_c_off[i] = -1
        continue
    rx = (map2[i] + (rgb_p.shift_m / z)) * rgb_p.fx + rgb_p.cx
    cx = rx
    cy = map4[i]
    c_off = cx + cy * 1920
    
    if c_off < 0 or c_off >= size_color:
        map_c_off[i] = -1
        continue
    
    map_c_off[i] = c_off

In [12]:
map_c_off[13333: 13416]

array([32961.12415108, 32963.94928836, 32966.48193171, 32969.07424988,
       32972.24163964, 32975.35851219, 32978.29029593, 32981.16880174,
       32983.94192218, 32986.63812745, 32989.90862858, 32992.97127539,
       32995.66369191, 32998.46266676, 33001.3660438 , 33004.2951711 ,
       33007.14610254, 33010.10058857, 33012.84805428, 33015.95683762,
       33018.91050648, 33021.760123  , 33024.53260395, 33027.45946361,
       33030.38604602, 33033.36376548, 33036.26407366, 33039.21555641,
       33042.03835648, 33045.11781262, 33048.04276042, 33050.96744061,
       33053.78888938, 33056.63617909, 33059.40727507, 33062.33089858,
       33065.22891606, 33068.17736479, 33071.27858511, 33074.17558876,
       33077.09792241, 33079.84186729, 33082.9674148 , 33085.76140909,
       33088.63194277, 33091.50236439, 33094.42319566, 33097.26805075,
       33100.01286473, 33102.95793847, 33105.82787885, 33108.69771192,
       33111.56743854, 33114.36302508, 33117.23273712, 33120.05341055,
      

In [13]:
registered_data = []

for i in range(size_depth):
    c_off = int(map_c_off[i])
    if c_off >= 0:
        registered_data.append(color_flatten[c_off])
    else:
        registered_data.append([0,0,0])

In [14]:
registered_data = np.asarray(registered_data)

In [15]:
registered_data = registered_data.reshape(424, 512, 3)

In [16]:
import cv2

img = registered_data.astype(np.uint8)

#cv2.imshow('Test', img)
#k = cv2.waitKey()

In [17]:
cv2.imwrite('/home/commaai-03/Test/tmp/data/test.png', img)

True

In [ ]:
registered_data

In [ ]:
def distort(x, y, intrinsic, method=0):
    '''Undistortion Image(depth).
    method:
    0: DISTORTION_INVERSE_BROWN_CONRADY
    1: DISTORTION_KANNALA_BRANDT4
    2: DISTORTION_FTHETA
    '''
    k1, k2, p1, p2, k3 = [intrinsic.k1,
                          intrinsic.k2,
                          intrinsic.p1,
                          intrinsic.p2,
                          intrinsic.k3]
    
    if type(method) == str:
        method = method.upper()
    if method == 'DISTORTION_INVERSE_BROWN_CONRADY' or method == 0:
        r2 = x * x + y * y
        f = 1 + k1 * r2 + k2 * r2 ** 2 + k3 * r2 ** 3
        ux = x * f + 2 * p1 * x * y + p2 * (r2 + 2 * x * x)
        uy = y * f + 2 * p2 * x * y + p1 * (r2 + 2 * y * y)
        return ux, uy
    if method == 'DISTORTION_KANNALA_BRANDT4' or method == 1:
        import math, sys
        rd = math.sqrt(x * x + y * y)
        eps = sys.float_info.epsilon
        # Avoid rd == 0
        if rd < eps:
            rd = eps
        theta = rd
        theta2 = rd * rd
        for i in range(4):
            f = theta * (1 + theta2 * (k1 + theta2 * (k2 + theta2 * (p1 + theta2 * p2)))) - rd
            if abs(f) < eps:
                break
            df = 1 + theta2 * (3 * k1 + theta2 * (5 * k2 + theta2 * (7 * p1 + 9 * theta2 * p2)))
            theta -= f / df
            theta2 = theta * theta
        
        r = math.tan(theta)
        x *= r / rd
        y *= r / rd
        ux, uy = x, y
        return ux, uy
    if method == 'DISTORTION_FTHETA' or method == 2:
        import math, sys
        rd = math.sqrt(x * x + y * y)
        eps = sys.float_info.epsilon
        # Avoid rd == 0
        if rd < eps:
            rd = eps
        r = math.tan(k1 * rd) / math.atan(2 * math.tan(k1 / 2.0))
        x *= r / rd
        y *= r / rd
        ux, uy = x, y
        return ux, uy

def deproject_pixel_to_point(pixel, depth, intrinsic, undistort=False):
    '''Calculate pixel and depth with no distortion
    to 3D coordinate of this camera's view.
    Args:
    ------
    intrinsic: camera intrinsic.
    pixel: pixel coordinate on depth.
    depth: the depth value of the pixel coordinate.
    '''
    
    x = (pixel[0] - intrinsic.ppx) / intrinsic.fx
    y = (pixel[1] - intrinsic.ppy) / intrinsic.fy
    ux, uy = x, y
    if undistort:
        ux, uy = distort(x, y, intrinsic, method=0)
    point = np.empty(3)
    point[0] = depth * ux
    point[1] = depth * uy
    point[2] = depth
    
    return point

def transform_p2p(from_point, rotation=None, translation=None, extrinsic=None):
    '''Transform 3D coordinates to other coordinate.
    Args:
    ------
    from_point: ndarray, (3,)
    rotation: ndarray, (3, 3)
    translation: ndarray, (3,)
    extrinsic(optional): include rotation, translation.
    '''
    
    if extrinsic:
        rotation = extrinsic.rotation
        translation = extrinsic.translation
    assert rotation.shape == (3, 3)
    assert (translation.shape == (3,) or translation == (3,1))
    translation = translation.reshape(3)
    to_point = np.empty(3)
    to_point = np.dot(rotation, from_point) + translation
    
    return to_point

def distort2(x, y, intrinsic, method=0):
    
    k1, k2, p1, p2, k3 = [intrinsic.k1,
                          intrinsic.k2,
                          intrinsic.p1,
                          intrinsic.p2,
                          intrinsic.k3]
    
    r2 = x * x + y * y
    if method == 0:
        f = 1 + k1 * r2 + k2 * r2 ** 2 + k3 * r2 ** 3
        dx = x * f + 2 * p1 * x * y + p2 * (r2 + 2 * x * x)
        dy = y * f + 2 * p2 * x * y + p1 * ( r2 + 2 * y * y)
        return dx, dy
    if method == 1:
        import math, sys
        r = math.sqrt(r2)
        eps = sys.float_info.epsilon
        if r < eps:
            r = eps
        theta = math.atan(r)
        theta2 = theta * theta
        series = 1 + theta2 * (k1 + theta2 * (k2 + theta2 * (p1 + theta2 * p2)))
        rd = theta * series
        x *= rd / r
        y *= rd / r
        dx, dy = x, y
        return dx, dy
    if method == 2:
        import math, sys
        r = math.sqrt(r2)
        eps = sys.float_info.epsilon
        if r < eps:
            r = eps
        rd = 1.0 / k1 * math.atan(2 * r * math.tan(k1 / 2.0))
        x *= rd / r
        y *= rd / r
        dx, dy = x, y
        return dx, dy
        
def project_point_to_pixel(point, intrinsic, method=0, undistort=False):
    
    assert point[2] != 0
    x, y = point[0] / point[2], point[1] / point[2]
    x, y = distort2(x, y, intrinsic, method=method)
    return x, y